In [41]:
import pandas as pd
import mysql.connector
import sqlite3
import matplotlib.pyplot as plt
import numpy as np

conn = mysql.connector.connect(
    host = 'localhost',
    user = 'root',
    password = 'ZeusInHades@90',
    database = 'sales'
)

df = pd.read_sql(
            "SELECT o.orderID,\
            companyName as 'Customer Name',\
            concat_ws(' ', firstName, lastName) as 'Employee Name',\
            concat('$',round(sum(Quantity * p.UnitPrice),2)) as 'Purchasing Price',\
            concat('$',round(sum(od.UnitPrice * Quantity),2)) as 'Selling Price',\
            round(sum(abs(od.UnitPrice * Quantity - Quantity * p.UnitPrice)),2) as 'Absolute Profit/Loss',\
            (case\
                when sum(od.UnitPrice * Quantity - Quantity * p.UnitPrice) < 0\
                then concat('Loss of ', concat('$',round(sum(abs(od.UnitPrice * Quantity - Quantity * p.UnitPrice)),2)))\
                when sum(od.UnitPrice * Quantity - Quantity * p.UnitPrice) = 0\
                then 'No Profit / Loss'\
                else concat('Profit of ', concat('$',round(sum(abs(od.UnitPrice * Quantity - Quantity * p.UnitPrice)),2)))\
                end) as Remark\
            from employees\
            join orders o using(employeeID)\
            join customers using(customerID)\
            join orderdetails od on o.orderID = od.orderID\
            join products p using(ProductID)\
            group by od.orderID \
            order by o.orderID asc", conn
    )

C:\Users\Rafael\AppData\Local\Temp\ipykernel_16496\1378196057.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


In [3]:
%matplotlib inline

In [45]:
df

,orderID,Customer Name,Employee Name,Purchasing Price,Selling Price,Absolute Profit/Loss,Remark
0,10248,Vins et alcools Chevalier,Steven Buchanan,$566.00,$440.00,126.00,Loss of $126.00
1,10249,Toms Spezialitten,Michael Suyama,$2329.25,$1863.40,465.85,Loss of $465.85
2,10250,Hanari Carnes,Margaret Peacock,$2267.25,$1813.00,454.25,Loss of $454.25
3,10251,Victuailles en stock,Janet Leverling,$839.50,$670.80,168.70,Loss of $168.70
4,10252,Suprmes dlices,Margaret Peacock,$4662.50,$3730.00,932.50,Loss of $932.50
...,...,...,...,...,...,...,...
825,11073,Pericles Comidas clsicas,Andrew Fuller,$300.00,$300.00,0.00,No Profit / Loss
826,11074,Simons bistro,Robert King,$244.30,$244.30,0.00,No Profit / Loss
827,11075,Richter Supermarkt,Laura Callahan,$586.00,$586.00,0.00,No Profit / Loss
828,11076,Bon app',Margaret Peacock,$1057.00,$1057.00,0.00,No Profit / Loss


In [43]:
df.describe()

,orderID,Absolute Profit/Loss
count,830.000000,830.000000
mean,10662.500000,114.347855
std,239.744656,306.039738
min,10248.000000,0.000000
25%,10455.250000,0.000000
50%,10662.500000,0.000000
75%,10869.750000,79.650000
max,11077.000000,3072.400000


In [24]:
df.loc[0:3]

,orderID,Customer Name,Employee Name,Purchasing Price,Selling Price,Absolute Profit/Loss,Remark
0,10248,Vins et alcools Chevalier,Steven Buchanan,$ 566.00,$ 440.00,126.00,Loss of $ 126.00
1,10249,Toms Spezialitten,Michael Suyama,$ 2329.25,$ 1863.40,465.85,Loss of $ 465.85
2,10250,Hanari Carnes,Margaret Peacock,$ 2267.25,$ 1813.00,454.25,Loss of $ 454.25
3,10251,Victuailles en stock,Janet Leverling,$ 839.50,$ 670.80,168.70,Loss of $ 168.70


In [25]:
df.iloc[0:3]

,orderID,Customer Name,Employee Name,Purchasing Price,Selling Price,Absolute Profit/Loss,Remark
0,10248,Vins et alcools Chevalier,Steven Buchanan,$ 566.00,$ 440.00,126.00,Loss of $ 126.00
1,10249,Toms Spezialitten,Michael Suyama,$ 2329.25,$ 1863.40,465.85,Loss of $ 465.85
2,10250,Hanari Carnes,Margaret Peacock,$ 2267.25,$ 1813.00,454.25,Loss of $ 454.25


In [44]:
df['Absolute Profit/Loss'] < 200

0       True
1      False
2      False
3       True
4      False
       ...  
825     True
826     True
827     True
828     True
829     True
Name: Absolute Profit/Loss, Length: 830, dtype: bool

In [52]:
df.loc[df['Absolute Profit/Loss'] > 200,\
       ['Customer Name', 'Employee Name', 'Purchasing Price', 'Selling Price', 'Absolute Profit/Loss', 'Remark']]

,Customer Name,Employee Name,Purchasing Price,Selling Price,Absolute Profit/Loss,Remark
1,Toms Spezialitten,Michael Suyama,$2329.25,$1863.40,465.85,Loss of $465.85
2,Hanari Carnes,Margaret Peacock,$2267.25,$1813.00,454.25,Loss of $454.25
4,Suprmes dlices,Margaret Peacock,$4662.50,$3730.00,932.50,Loss of $932.50
5,Hanari Carnes,Janet Leverling,$1806.00,$1444.80,361.20,Loss of $361.20
7,Richter Supermarkt,Anne Dodsworth,$3115.75,$2490.50,625.25,Loss of $625.25
...,...,...,...,...,...,...
240,Frankenversand,Laura Callahan,$1950.00,$1560.00,390.00,Loss of $390.00
242,HILARION-Abastos,Robert King,$3954.00,$3163.20,790.80,Loss of $790.80
244,Bottom-Dollar Markets,Janet Leverling,$1120.00,$896.00,224.00,Loss of $224.00
246,Comrcio Mineiro,Margaret Peacock,$1140.00,$912.00,228.00,Loss of $228.00


In [64]:
df[['Customer Name', 'Absolute Profit/Loss']]

,Customer Name,Absolute Profit/Loss
0,Vins et alcools Chevalier,126.00
1,Toms Spezialitten,465.85
2,Hanari Carnes,454.25
3,Victuailles en stock,168.70
4,Suprmes dlices,932.50
...,...,...
825,Pericles Comidas clsicas,0.00
826,Simons bistro,0.00
827,Richter Supermarkt,0.00
828,Bon app',0.00
